# The spaceship operator

## Motivation


There are frequent situations where one need some `<` operator for a home-made class. Typically, if you want to make a `std::vector` of such objects, and sort it. But also if you want to use it as a key for some `std::set` or `std::map`.

For completeness, one will also add `>`, `>=`, and `<=`, implemented as a reuse of either `<` and `==`, or `<` and `>`.

Those operators will be defined as an external functions, optionally friends, so that left and right arguments will be similarly convertibles.

Much boilerplate code to write. Too much.

## Idea

C++20 introduce the new *spaceship* operator : `<=>`. Well, the real official name is *three-way comparison operator*.

It is provided by default for all predefined types, and return *something* which can be compared to `0` (such as the `std::strcmp`), meaning *lower than* if this *something* is lower than `0`, *greater than* if it is greater than `0`, and *equivalent* if it is equal `0`:

In [ ]:
#include <iostream>

template <typename T>
void three_way_compare( T lhs, T rhs )
 {
  auto result = (lhs<=>rhs) ;
  std::cout
    <<lhs<<" <=> "<<rhs<<": "<<'<'
    <<(result<0)<<'|'<<(result==0)<<'|'<<(result>0)
    <<'>'<<std::endl ;
 }

int main()
 {
  three_way_compare(1,2) ;
  three_way_compare(2,2) ;
  three_way_compare(2,1) ;
 }

## Different kinds of ordering

The real return type of `<=>` for integers is `std::strong_ordering` : whatever the values, you will always get `true` for exactly one test among `<0`, `==0`, and `>0`.

On the contrary, the return type of `<=>` for floating point numbers is `std::partial_ordering`, because sometimes the tree tests may return `false`, typically if one one the number is `NaN`.

In [ ]:
#include <iostream>

template <typename T>
void three_way_compare( T lhs, T rhs )
 {
  auto result = (lhs<=>rhs) ;
  std::cout
    <<lhs<<" <=> "<<rhs<<": "<<'<'
    <<(result<0)<<'|'<<(result==0)<<'|'<<(result>0)
    <<'>'<<std::endl ;
 }

int main()
 {
  three_way_compare(+0.,-0.) ;
  three_way_compare(0./1.,1./0.) ;
  three_way_compare(0.,0./0.) ;
 }

The type `std::strong_ordering` is convertible to `std::partial_ordering`, but not the contrary.

Between the two, we have a class `std::weak_ordering`, where `==0` mean that the two compared values are equivalent from a ranking point of view, but not necessarily *equal*. In a given expression, one cannot subsitutes one value for the other and be sure to have the same result.

I am not aware of some predefined type whose `<=>` would return an instance of ``std::weak_ordering`, but it may make sense for some home-made class, such as the following.

## Home-made class

In the example below, we define a very basic class for positive rational numbers, and provide an implementation of `<=>`:

In [ ]:
#include <iostream>
#include <compare>

struct Rational
 {
  unsigned n, d ;
  
  std::weak_ordering operator<=>( Rational const & other ) const
   { return (n*other.d)<=>(other.n*d) ; }
 } ;
 
std::ostream & operator<<( std::ostream & os, std::weak_ordering cmp )
 { return (os<<'<'<<(cmp<0)<<'|'<<(cmp==0)<<'|'<<(cmp>0)<<'>') ; }

std::ostream & operator<<( std::ostream & os, Rational const & r )
 { return (os<<r.n<<'/'<<r.d) ; }

template <typename T>
void compare( T lhs, T rhs )
 {
  std::cout<<std::endl ;
  //std::cout<<lhs<<" ==  "<<rhs<<": "<<(lhs==rhs)<<std::endl ;
  //std::cout<<lhs<<" !=  "<<rhs<<": "<<(lhs!=rhs)<<std::endl ;
  std::cout<<lhs<<" <=> "<<rhs<<": "<<(lhs<=>rhs)<<std::endl ;
  std::cout<<lhs<<" <   "<<rhs<<": "<<(lhs<  rhs)<<std::endl ;
  std::cout<<lhs<<" <=  "<<rhs<<": "<<(lhs<= rhs)<<std::endl ;
  std::cout<<lhs<<" >   "<<rhs<<": "<<(lhs>  rhs)<<std::endl ;
  std::cout<<lhs<<" >=  "<<rhs<<": "<<(lhs>= rhs)<<std::endl ;
 }

int main()
 {  
  compare<Rational>({ 3, 4 },{ 2, 3 }) ;
  compare<Rational>({ 3, 6 },{ 2, 3 }) ;
  compare<Rational>({ 1, 2 },{ 2, 4 }) ;
 }

Despites the `<=>` for `unsigned` returns some instance of `std::strong_ordering`, we prefer here to cast it to `std::weak_ordering`. This way, we emphasize that if `a<=>b` is equal to `0`, it only means that `a` and `b` are logically equivalent, but may lead to different results in some expressions.

As one can check in the code, we have defined `<=>`, and the compiler has added, for free, some default implementations for `<`, `>`, `<=` and `=>`. Of course, one can overload them with home-made implementations.

It has NOT added a default implementation for `==` and `!=`. Those operators are generally expected to mean *equal*, rather than *equivalent*. If `<=>` does not enable a strong order, the operator `==`generally deserves a separate implementation.

## Default `<=>` implementation

One can ask the compiler to provide some default implementation for `<=>`. Logically enough, it will compare the first member variable of the two objects, and goes on to the next member variable as long as the current ones are equivalent.

In the previous example, that would not be relevant, because it will compare the numerators first, and conclude that `3/6` is greater than `2/3`:

In [6]:
#include <iostream>

struct Rational
 {
  unsigned n, d ;
  std::strong_ordering operator<=>( Rational const & other ) const = default ;
 } ;
 
std::ostream & operator<<( std::ostream & os, std::weak_ordering cmp )
 { return (os<<'<'<<(cmp<0)<<'|'<<(cmp==0)<<'|'<<(cmp>0)<<'>') ; }
 
std::ostream & operator<<( std::ostream & os, Rational const & r )
 { return (os<<r.n<<'/'<<r.d) ; }

int main()
 {  
  Rational r1 { 3, 6 }, r2 { 2, 3 } ;
  std::cout<<r1<<" <=> "<<r2<<": "<<(r1<=>r2)<<std::endl ;
 }

SyntaxError: invalid syntax (2540279673.py, line 3)

On the contrary, for some tuple-like class, it may make sense:

In [ ]:
#include <iostream>
#include <string>
#include <set>

struct Grade
 {
  double number ;
  char letter ;
  std::string name ;
  
  auto operator<=>( Grade const & other ) const = default ;
 } ;
 
std::ostream & operator<<( std::ostream & os, Grade const & g )
 { return (os<<g.letter<<", "<<g.number<<", "<<g.name) ; }

int main()
 {
  std::set<Grade> grades
   {
     { 19, 'A', "Djamila" },
     { 12, 'C', "Charles" },
     { 16.5, 'A', "Marc" },
   } ;
  
  for ( auto const & grade : grades )
   { std::cout<<grade<<std::endl ; }
 }

We see above that the definition of `<=>` has been completed by the compiler with a default implementation of `<` (among others), which has been used by `std::set` so to sorts its elements.

Worth to note : if you use the *default* implementation of `<=>`, you will also get additional `==` and `!=`. Not sure thi sspecial behavior is such a good idea...

## What to retain

- Defining `<=>` brings you a free default implementation of `<`, `>`, `<=`, and `>=`.
- The standard library defines few different kinds of order (strong, weak and partial).
- Do not confuse equivalence (`(a<=>b)==0`) with equality (`a==b`).

# Questions ?

# Exercise

We are not fully happy with the default implementation of `<=>`for our `Complex` class below.
1. Modify it so that the ordering is based on the norm of the complexes.
2. Because you do not use any more the default implementation of `<=>`m you had to prived also an implementation for `==`. What happens if you deduce it from `<=>` (using `(((*this)<=>other)==0)`) ?
3. Try to restore the default implementation for `==` only.

In [2]:
%%file tmp.spaceship.cpp

#include <iostream>
#include <cmath>

struct Complex
 {
  double r, i ;
  double norm() const { return std::sqrt(r*r+i*i) ; }
  auto operator<=>( Complex const & other ) const = default ;
 } ;
 
std::ostream & operator<<( std::ostream & os, std::partial_ordering cmp )
 { return (os<<'<'<<(cmp<0)<<'|'<<(cmp==0)<<'|'<<(cmp>0)<<'>') ; }

std::ostream & operator<<( std::ostream & os, Complex const & c )
 { return (os<<c.r<<'+'<<c.i<<'i') ; }

template <typename T>
void compare( T lhs, T rhs )
 {
  std::cout<<std::endl ;
  std::cout<<lhs<<" ==  "<<rhs<<" : "<<(lhs==rhs)<<std::endl ;
  std::cout<<lhs<<" <=> "<<rhs<<" : "<<(lhs<=>rhs)<<std::endl ;
 }

int main()
 {  
  compare<Complex>({ 1., 2. },{ 1.5, 1.5 }) ;
  compare<Complex>({ 1., 0. },{ 0., 1. }) ;
 }

Writing tmp.spaceship.cpp


In [4]:
!rm -f tmp.spaceship.exe && g++ -std=c++20 tmp.spaceship.cpp -o tmp.spaceship.exe

In [5]:
!./tmp.spaceship.exe


1+2i ==  1.5+1.5i : 0
1+2i <=> 1.5+1.5i : <1|0|0>

1+0i ==  0+1i : 0
1+0i <=> 0+1i : <0|0|1>


# Sources

- https://blog.tartanllama.xyz/spaceship-operator/
- https://iq.opengenus.org/spaceship-operator-cpp/
- https://www.jonathanmueller.dev/talk/meetingcpp2019/

© *CNRS 2022*  
*Assembled and written by David Chamont, this work is made available according to the terms of the*  
[*Creative Commons License - Attribution - NonCommercial - ShareAlike 4.0 International*](http://creativecommons.org/licenses/by-nc-sa/4.0/)